In [1]:
%pip install --quiet -U pip -r ../requirements/requirements-training.txt ../.

Note: you may need to restart the kernel to use updated packages.


In [1]:
import warnings
warnings.filterwarnings("ignore")

from jumanji.training.train import train
from hydra import compose, initialize

In [17]:
env = "maze"  # @param ['bin_pack', 'cleaner', 'connector', 'cvrp', 'game_2048', 'job_shop', 'knapsack', 'maze', 'minesweeper', 'rubiks_cube', 'snake', 'sudoku', 'tsp']
agent = "random"  # @param ['random', 'a2c']

In [18]:
with initialize(version_base=None, config_path="../jumanji/training/configs"):
    cfg = compose(config_name="config.yaml", overrides=[f"env={env}", f"agent={agent}", "logger.type=terminal", "logger.save_checkpoint=true"])
cfg

In [ ]:
train(cfg)